# Options Chains Analysis

Version 3.1 of the OpenBB SDK provides a suite of enhancements to the Options module.  This notebook will highlight the changes and demonstrate how to get started using the new tools.

## What's Changed

- There's a new way to  `load`
  
        `openbb.stocks.options.load_options_chains()`

    The legacy method (`openbb.stocks.options.chains()`) remains in service, but some minor changes to existing scripts will be required to take advantage of the new functionality.  Don't worry, it's nothing that `Find & Replace` can't easily handle.  Let's compare below.

In [164]:
# Select the `obb` Python environment as the session's kernel, then import the OpenBB SDK.

from openbb_terminal.sdk import openbb
import pandas as pd
from openbb_terminal.economy.plot_view import show_plot

In [79]:
# The legacy method

spy_chains = openbb.stocks.options.chains("SPY")
spy_chains.head(2)

,optionType,expiration,strike,lastPrice,bid,ask,openInterest,volume
0,call,2023-06-21,350.0,87.11,84.94,85.11,2,0
1,put,2023-06-21,350.0,0.01,0.00,0.01,40,3


In [80]:
# The new method

spy = openbb.stocks.options.load_options_chains("SPY", "Nasdaq")
spy.chains.head(2)

,optionType,expiration,strike,lastPrice,bid,ask,openInterest,volume,dte
0,call,2023-06-21,350.0,87.11,84.94,85.11,2,0,0
1,put,2023-06-21,350.0,0.01,0.00,0.01,40,3,0


The same response (the options chains) is in both variables, but the new method contains more than just the chains data previously fetched.

In [81]:
spy.__dict__.keys()

dict_keys(['source', 'SYMBOLS', 'symbol', 'chains', 'expirations', 'strikes', 'underlying_price', 'last_price', 'underlying_name', 'hasIV', 'hasGreeks', 'get_available_greeks', 'date'])

The new object eliminates the need to make an additional request for the price of the underlying asset, or the list of expirations and strikes.

In [82]:
# Instead of:

last_price = openbb.stocks.options.price("SPY")

last_price

434.6

In [83]:
# Use the Options data object:

spy.last_price

434.6

- There are new sources of data

    Two additional data sources have been added, `CBOE` and `TMX`, with the latter covering the Canadian market.  The new default source is `CBOE`.  The DataFrame has been transposed below to better illustrate the additional columns available by default.

In [84]:
spy = openbb.stocks.options.load_options_chains("SPY")

spy.chains.query("`strike` == 435").head(6).transpose()

,148,149,418,419,726,727
expiration,2023-06-21,2023-06-21,2023-06-22,2023-06-22,2023-06-23,2023-06-23
strike,435.0,435.0,435.0,435.0,435.0,435.0
optionType,call,put,call,put,call,put
contractSymbol,SPY230621C00435000,SPY230621P00435000,SPY230622C00435000,SPY230622P00435000,SPY230623C00435000,SPY230623P00435000
bid,0.82,0.83,1.53,1.34,1.95,1.71
bidSize,61,59,54,49,41,88
ask,0.83,0.84,1.54,1.35,1.97,1.72
askSize,59,59,54,57,40,54
impliedVolatility,0.1914,0.1922,0.1421,0.1419,0.1344,0.1349
openInterest,4886,13470,954,7465,8003,27191


`TMX` provides historical EOD chains data, beginning in 2009.  Notably, this data contains implied volatility, the current quotes do not.

In [44]:
xiu = openbb.stocks.options.load_options_chains("XIU", "TMX", "2009-07-01")
print("The closing price of ", xiu.symbol,  " on, ", xiu.date, ", was: $", xiu.last_price, sep ="")
xiu.chains.query("`strike` == 15").head(6).transpose()

The closing price of XIU on, 2009-07-02, was: $15.59


,10,11,30,31,64,65
expiration,2009-07-17 00:00:00,2009-07-17 00:00:00,2009-08-21 00:00:00,2009-08-21 00:00:00,2009-09-18 00:00:00,2009-09-18 00:00:00
strike,15.0,15.0,15.0,15.0,15.0,15.0
optionType,call,put,call,put,call,put
date,2009-07-02,2009-07-02,2009-07-02,2009-07-02,2009-07-02,2009-07-02
bid,0.68,0.1,0.94,0.34,1.14,0.56
ask,0.77,0.15,1.05,0.43,1.26,0.63
bidSize,510,500,510,800,510,510
askSize,52,2,330,300,300,300
lastPrice,0.77,0.15,1.05,0.43,1.26,0.63
volume,0,60,0,0,0,15


In [85]:
xiu = openbb.stocks.options.load_options_chains("XIU", "TMX")
xiu.chains.query("30 <= `strike` <= 31").head(10)

,expiration,strike,optionType,bid,ask,lastPrice,change,openInterest,volume,dte
14,2023-06-23,30.00,call,0.06,0.10,0.20,0.0,30,0,2
15,2023-06-23,30.00,put,0.13,0.18,0.13,0.0,104,0,2
16,2023-06-23,30.25,call,0.00,0.09,0.10,0.0,0,0,2
17,2023-06-23,30.25,put,0.21,0.46,0.30,0.0,0,0,2
18,2023-06-23,30.50,call,0.00,0.08,0.09,0.0,49,0,2
19,2023-06-23,30.50,put,0.44,0.70,0.65,0.0,5,0,2
20,2023-06-23,30.75,call,0.00,0.08,0.09,0.0,51,0,2
21,2023-06-23,30.75,put,0.69,0.99,0.90,0.0,0,0,2
22,2023-06-23,31.00,call,0.00,0.08,0.08,0.0,0,0,2
23,2023-06-23,31.00,put,0.94,1.20,1.15,0.0,10,0,2


- Methods are bound to the returned object that sort and filter the data
    Along with attributes listed above - `spy.__dict__.keys()` - methods are provided for extracting key insights from the raw data.

    - `get_stats()`
    - `get_skew()`
    - `get_straddle()`
    - `get_strangle()`
    - `get_vertical_call_spread()`
    - `get_vertical_put_spread()`
    - `get_strategies()`

`impliedVolatility` is a required field for `get_skew()`; therefore, only sources returning `hasIV` as `True` will work.  All other callables are compatible with every source.


## Getting Started

Assign a ticker to a new variable.

In [86]:
ticker = openbb.stocks.options.load_options_chains("QQQ")
print(ticker.underlying_name, '\n', "Symbol: ", ticker.symbol, '\n', "Source: ", ticker.source, '\n', "Last Price: " , ticker.last_price, sep = "")

INVESCO QQQ TR UNIT SER 1
Symbol: QQQ
Source: CBOE
Last Price: 362.0


Let's look at volume and open interest, by expiration.

In [87]:
oi = ticker.get_stats()[["Puts OI", "Calls OI"]]
oi["Puts OI"] = oi["Puts OI"] * (-1)
vol = ticker.get_stats()[["Puts Volume", "Calls Volume"]]
vol["Puts Volume"] = vol["Puts Volume"] * (-1)

In [88]:
openbb.forecast.plot(oi, oi.columns.to_list())

In [89]:
openbb.forecast.plot(vol, vol.columns.to_list())

The concentration of volume is clearly at the front contract, while the open interest paints a different picture.  The same analysis can be easily applied by strike.


In [90]:
strike_oi = ticker.get_stats("strike")[["Puts OI", "Calls OI"]]
strike_oi["Puts OI"] = strike_oi["Puts OI"] * (-1)
strike_vol = ticker.get_stats("strike")[["Puts Volume", "Calls Volume"]]
strike_vol["Puts Volume"] = strike_vol["Puts Volume"] * (-1)

In [68]:
openbb.forecast.plot(strike_oi, strike_oi.columns.to_list())

In [74]:
openbb.forecast.plot(strike_vol, strike_vol.columns.to_list())

Delta exposure can easily be calculated by multiplying the delta value in the chains data by (OI * 100 * share price)

In [132]:
df = ticker.chains[["expiration", "dte", "strike", "optionType", "openInterest", "delta"]]
df["Delta Exposure"] = df["delta"] * df["openInterest"] * (100) * ticker.last_price
dex = pd.DataFrame()
dex["Call DEX"] = df.query("`optionType` == 'call'").set_index(["expiration","strike"])[["Delta Exposure"]]
dex["Put DEX"] = df.query("`optionType` == 'put'").set_index(["expiration","strike"])[["Delta Exposure"]]

# By expiration date

dex_by_expiry = dex.reset_index().groupby("expiration")[["Call DEX", "Put DEX"]].sum()

openbb.forecast.plot(dex_by_expiry, dex_by_expiry.columns.to_list())

In [133]:
# By strike price

dex_by_strike = dex.reset_index().groupby("strike")[["Call DEX", "Put DEX"]].sum()

openbb.forecast.plot(dex_by_strike, dex_by_strike.columns.to_list())

The Put/Call Ratios can be visualized using columns returned by, `get_stats()`

In [165]:
itm = ticker.get_stats()[["ITM Percent"]]
pcr = ticker.get_stats()[["OI Ratio"]]
pcr_fig = show_plot(itm, pcr, external_axes = True)
pcr_fig.update_layout(
    xaxis = {'title': 'Expiration Date'},
    yaxis= {'title': '% of OI ITM'},
    yaxis2 = {'title': 'Put/Call Ratio'},
    title =  f"{ticker.symbol} - Put/Call Ratio vs % ITM",
    title_y = 0.98,
    title_x = 0.5,
    legend=dict(
        yanchor="top",
        y=1,
        xanchor="right",
        x=0.9
    )
)
pcr_fig.show()

Make a chart of the term structure using the implied volatility skew, or the cost of an at-the-money straddle.

In [199]:
skew = ticker.get_skew(moneyness=10)[["ATM Skew", "IV Skew"]].rename(columns = {"IV Skew": "10% OTM Skew"})
y1 = pd.DataFrame(skew["ATM Skew"])
y2 = pd.DataFrame(skew["10% OTM Skew"])
skew_fig = show_plot(y1,y2, external_axes = True)
skew_fig.update_layout(
    xaxis = {'title': 'Expiration Date'},
    yaxis= {'title': 'ATM IV Skew'},
    yaxis2 = {'title': '10% OTM IV Skew'},
    title =  f"{ticker.symbol} - ATM Skew vs 10% OTM Skew",
    title_y = 0.98,
    title_x = 0.5,
    legend=dict(
        yanchor="bottom",
        y=0.1,
        xanchor="right",
        x=0.9
    )
)
skew_fig.update_yaxes(title_font=dict(size=20), tickfont=dict(size=12))
skew_fig.show()

In [196]:
y1 = pd.DataFrame(ticker.get_strategies().set_index("Expiration")["Cost Percent"])
y2 = pd.DataFrame(skew[["10% OTM Skew"]])
skew_fig2 = show_plot(y1,y2, external_axes = True)
skew_fig2.update_layout(
    xaxis = {'title': 'Expiration Date'},
    yaxis= {'title': 'Cost as % of Stock Price'},
    yaxis2 = {'title': '10% OTM IV Skew'},
    title =  f"{ticker.symbol} - Cost of ATM Straddle vs 10% OTM Skew",
    title_y = 0.98,
    title_x = 0.5,
    legend=dict(
        yanchor="bottom",
        y=0.1,
        xanchor="right",
        x=0.9
    )
)
skew_fig2.update_yaxes(title_font=dict(size=16))
skew_fig2.show()

No strategy was selected, returning all ATM straddles.


Basic single-leg strategies can be screened via `get_strategies()`.  Values entered for each argument return the closest match.  With no parameters specified, all ATM straddles are returned.

In [202]:
ticker.get_strategies(days = [5,30], straddle_strike = ticker.last_price, strangle_moneyness = [10,-20] , vertical_calls = [365,367], vertical_puts = [360,358])

,Expiration,DTE,Strategy,Underlying Price,Strike 1,Strike 2,Strike 1 Premium,Strike 2 Premium,Cost,Cost Percent,Breakeven Upper,Breakeven Upper Percent,Breakeven Lower,Breakeven Lower Percent,Max Profit,Max Loss,Payoff Ratio
0,2023-06-26,5,Long Straddle,362.0,362.0,362.0,3.04,2.94,5.98,1.6519,367.98,1.6519,356.02,-1.6519,inf,-5.98,inf
1,2023-06-26,5,Long Strangle,362.0,398.0,326.0,0.01,0.03,0.04,0.011,398.04,9.9558,325.96,-9.9558,inf,-0.04,inf
2,2023-06-26,5,Bear Call Spread,362.0,365.0,367.0,1.74,1.17,-0.57,-0.1575,365.57,1.2238,NaN,NaN,0.57,-1.43,0.3986
3,2023-06-26,5,Bull Put Spread,362.0,360.0,358.0,2.07,1.44,-0.63,0.174,359.37,0.7265,NaN,NaN,0.63,-1.37,0.4599
4,2023-07-21,30,Long Straddle,362.0,362.0,362.0,8.34,6.95,15.29,4.2238,377.29,4.2238,346.71,-4.2238,inf,-15.29,inf
5,2023-07-21,30,Long Strangle,362.0,398.0,326.0,0.32,0.7,1.02,0.2818,399.02,10.2265,324.98,-10.2265,inf,-1.02,inf
6,2023-07-21,30,Short Strangle,362.0,435.0,290.0,0.0,0.13,-0.13,0.0359,435.13,20.2017,289.87,-19.9254,0.13,inf,0.0
7,2023-07-21,30,Bear Call Spread,362.0,365.0,367.0,6.73,5.83,-0.9,-0.2486,365.9,1.1326,NaN,NaN,0.9,-1.1,0.8182
8,2023-07-21,30,Bull Put Spread,362.0,360.0,358.0,6.07,5.33,-0.74,0.2044,359.26,0.7569,NaN,NaN,0.74,-1.26,0.5873


In [201]:
ticker.get_strategies()

No strategy was selected, returning all ATM straddles.


,Expiration,DTE,Strategy,Underlying Price,Strike 1,Strike 2,Strike 1 Premium,Strike 2 Premium,Cost,Cost Percent,Breakeven Upper,Breakeven Upper Percent,Breakeven Lower,Breakeven Lower Percent,Max Profit,Max Loss,Payoff Ratio
0,2023-06-22,1,Long Straddle,362.0,362.0,362.0,1.95,1.93,3.88,1.0718,365.88,1.0718,358.12,-1.0718,inf,-3.88,inf
1,2023-06-23,2,Long Straddle,362.0,362.0,362.0,2.52,2.45,4.97,1.3729,366.97,1.3729,357.03,-1.3729,inf,-4.97,inf
2,2023-06-26,5,Long Straddle,362.0,362.0,362.0,3.04,2.94,5.98,1.6519,367.98,1.6519,356.02,-1.6519,inf,-5.98,inf
3,2023-06-27,6,Long Straddle,362.0,362.0,362.0,3.46,3.3,6.76,1.8674,368.76,1.8674,355.24,-1.8674,inf,-6.76,inf
4,2023-06-28,7,Long Straddle,362.0,362.0,362.0,3.84,3.63,7.47,2.0635,369.47,2.0635,354.53,-2.0635,inf,-7.47,inf
5,2023-06-29,8,Long Straddle,362.0,362.0,362.0,4.24,3.91,8.15,2.2514,370.15,2.2514,353.85,-2.2514,inf,-8.15,inf
6,2023-06-30,9,Long Straddle,362.0,362.0,362.0,4.64,4.22,8.86,2.4475,370.86,2.4475,353.14,-2.4475,inf,-8.86,inf
7,2023-07-03,12,Long Straddle,362.0,362.0,362.0,4.98,4.48,9.46,2.6133,371.46,2.6133,352.54,-2.6133,inf,-9.46,inf
8,2023-07-05,14,Long Straddle,362.0,362.0,362.0,5.34,4.77,10.11,2.7928,372.11,2.7928,351.89,-2.7928,inf,-10.11,inf
9,2023-07-07,16,Long Straddle,362.0,362.0,362.0,5.92,5.2,11.12,3.0718,373.12,3.0718,350.88,-3.0718,inf,-11.12,inf


It's now easier than ever to analyze equity options.  See the introduction guide [here](https://docs.openbb.co/sdk/usage/intros/stocks/options-chains) for more information on using these functions.